In [34]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torchvision
from pathlib import Path
import time
import copy
import shutil

In [35]:
torch.__version__

'2.9.0+cu128'

In [36]:
input_path = Path("data/datasets/trashnet_01/")
input_path2 = Path("data/datasets/self-collected_outdoor/")

In [37]:
import random

seed = 16

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [38]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [39]:
from collections import Counter

def compute_class_weights_from_imagefolder(train_root, num_classes):
    ds = datasets.ImageFolder(train_root, transform=transforms.ToTensor())
    counts = Counter(ds.targets)
    total = sum(counts.values())

    weights = [total / counts[i] for i in range(num_classes)]
    weights = torch.tensor(weights, dtype=torch.float)
    weights = weights / weights.sum() * num_classes 
    return weights, ds.classes, counts

tmp_ds = datasets.ImageFolder(input_path / "train")
num_classes = len(tmp_ds.classes)

class_weights, class_names, train_counts = compute_class_weights_from_imagefolder(
    input_path / "train",
    num_classes
)

class_weights = class_weights.to(device)

criterion_weighted = torch.nn.CrossEntropyLoss(weight=class_weights)

print("Classes:", class_names)
print("Train counts:", train_counts)
print("Class weights:", class_weights)

Classes: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']
Train counts: Counter({3: 475, 1: 400, 4: 385, 2: 328, 0: 322, 5: 109})
Class weights: tensor([0.8270, 0.6657, 0.8119, 0.5606, 0.6917, 2.4431], device='cuda:0')


In [40]:
# baseline data transforms

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

data_transforms = {

    "train": transforms.Compose([
        transforms.RandomResizedCrop(224),  
        transforms.RandomHorizontalFlip(), 
        transforms.ToTensor(),
        normalize
    ]),
    "val": transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),    
}

image_datasets = {
    "train": datasets.ImageFolder(input_path / "train", data_transforms["train"]),
    "val": datasets.ImageFolder(input_path / "val", data_transforms["val"]),
    "test": datasets.ImageFolder(input_path / "test", data_transforms["val"]),

    "test_01": datasets.ImageFolder(input_path2, data_transforms["val"])
}


dataloaders = {
    "train": torch.utils.data.DataLoader(image_datasets["train"], batch_size=32, shuffle=True, num_workers=4),
    "val": torch.utils.data.DataLoader(image_datasets["val"], batch_size=32, shuffle=False, num_workers=4),
    "test": torch.utils.data.DataLoader(image_datasets["test"], batch_size=32, shuffle=False, num_workers=4),

    "test_01": torch.utils.data.DataLoader(image_datasets["test_01"], batch_size=32, shuffle=False, num_workers=4)
}   


dataset_size = {
    "train": len(image_datasets["train"]),
    "val": len(image_datasets["val"]),
    "test_01": len(image_datasets["test_01"])
}

num_classes = len((image_datasets["train"]).classes)

In [41]:
#geometric augmentation

data_transforms_geo = {
    "train": transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.RandomAffine(degrees=0, translate=(0.05, 0.05), scale=(0.9, 1.1), shear=5),
        transforms.ToTensor(),
        normalize
    ]),
    "val": transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]), 
}

image_datasets_geo = {
    "train": datasets.ImageFolder(input_path / "train", data_transforms_geo["train"]),
    "val": datasets.ImageFolder(input_path / "val", data_transforms_geo["val"]),
    "test": datasets.ImageFolder(input_path / "test", data_transforms_geo["val"]),
    "test_01": datasets.ImageFolder(input_path2, data_transforms_geo["val"])
}

dataloaders_geo = {
    "train": torch.utils.data.DataLoader(image_datasets_geo["train"], batch_size=32, shuffle=True, num_workers=4),
    "val": torch.utils.data.DataLoader(image_datasets_geo["val"], batch_size=32, shuffle=False, num_workers=4),
    "test": torch.utils.data.DataLoader(image_datasets_geo["test"], batch_size=32, shuffle=False, num_workers=4),
    "test_01": torch.utils.data.DataLoader(image_datasets_geo["test_01"], batch_size=32, shuffle=False, num_workers=4)
}

dataset_size_geo = {
    "train": len(image_datasets_geo["train"]),
    "val": len(image_datasets_geo["val"]),
    "test": len(image_datasets_geo["test"]),
    "test_01": len(image_datasets_geo["test_01"])
}

num_classes_geo = len(image_datasets_geo["train"].classes)


In [42]:
#photometric
 
data_transforms_photo = {
    "train": transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1),
        transforms.RandomGrayscale(p=0.1),
        transforms.ToTensor(),
        normalize
    ]),
    "val": transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]), 
}

image_datasets_photo = {
    "train": datasets.ImageFolder(input_path / "train", data_transforms_photo["train"]),
    "val": datasets.ImageFolder(input_path / "val", data_transforms_photo["val"]),
    "test": datasets.ImageFolder(input_path / "test", data_transforms_photo["val"]),
    "test_01": datasets.ImageFolder(input_path2, data_transforms_photo["val"])
}

dataloaders_photo = {
    "train": torch.utils.data.DataLoader(image_datasets_photo["train"], batch_size=32, shuffle=True, num_workers=4),
    "val": torch.utils.data.DataLoader(image_datasets_photo["val"], batch_size=32, shuffle=False, num_workers=4),
    "test": torch.utils.data.DataLoader(image_datasets_photo["test"], batch_size=32, shuffle=False, num_workers=4),
    "test_01": torch.utils.data.DataLoader(image_datasets_photo["test_01"], batch_size=32, shuffle=False, num_workers=4)
}

dataset_size_photo = {
    "train": len(image_datasets_photo["train"]),
    "val": len(image_datasets_photo["val"]),
    "test": len(image_datasets_photo["test"]),
    "test_01": len(image_datasets_photo["test_01"])
}

num_classes_photo = len(image_datasets_photo["train"].classes)


In [43]:
#aug mix
 
data_transforms_mix = {
    "train": transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.RandomAffine(degrees=0, translate=(0.05, 0.05), scale=(0.9, 1.1), shear=5),
        transforms.ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1),
        transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
        transforms.ToTensor(),
        normalize,
        transforms.RandomErasing(p=0.5, scale=(0.02, 0.2), ratio=(0.3, 3.3), value='random')
    ]),
    "val": transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]), 
}

image_datasets_mix = {
    "train": datasets.ImageFolder(input_path / "train", data_transforms_mix["train"]),
    "val": datasets.ImageFolder(input_path / "val", data_transforms_mix["val"]),
    "test": datasets.ImageFolder(input_path / "test", data_transforms_mix["val"]),
    "test_01": datasets.ImageFolder(input_path2, data_transforms_mix["val"])
}

dataloaders_mix = {
    "train": torch.utils.data.DataLoader(image_datasets_mix["train"], batch_size=32, shuffle=True, num_workers=4),
    "val": torch.utils.data.DataLoader(image_datasets_mix["val"], batch_size=32, shuffle=False, num_workers=4),
    "test": torch.utils.data.DataLoader(image_datasets_mix["test"], batch_size=32, shuffle=False, num_workers=4),
    "test_01": torch.utils.data.DataLoader(image_datasets_mix["test_01"], batch_size=32, shuffle=False, num_workers=4)
}

dataset_size_mix = {
    "train": len(image_datasets_mix["train"]),
    "val": len(image_datasets_mix["val"]),
    "test": len(image_datasets_mix["test"]),
    "test_01": len(image_datasets_mix["test_01"])
}

num_classes_mix = len(image_datasets_mix["train"].classes)


In [44]:
#MixUp function

def apply_mixup(inputs, labels, alpha=0.4):
    if alpha <= 0:
        return inputs, labels, labels, 1.0

    lam = np.random.beta(alpha, alpha)
    batch_size = inputs.size(0)
    index = torch.randperm(batch_size, device=inputs.device)

    mixed_inputs = lam * inputs + (1 - lam) * inputs[index]
    labels_a = labels
    labels_b = labels[index]
    return mixed_inputs, labels_a, labels_b, lam

In [45]:
def train_model(model, dataloaders, dataset_size, device, criterion, optimizer, num_epochs = 25, use_mixup=False, mixup_prob=0.5, mixup_alpha=0.4):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    history = {
        "train_loss": [],
        "train_acc": [],
        "val_loss": [],
        "val_acc": [],
    }

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print("-" * 10)

        for phase in ["train", "val"]:
            if phase == "train":
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0.0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == "train"):

                        if phase == "train" and use_mixup and (np.random.rand() < mixup_prob):
                            inputs_mixed, y_a, y_b, lam = apply_mixup(inputs, labels, alpha=mixup_alpha)

                            outputs = model(inputs_mixed)
                            _, preds = torch.max(outputs, 1)

                            loss = lam * criterion(outputs, y_a) + (1 - lam) * criterion(outputs, y_b)

                            running_corrects += (float(lam) * torch.sum(preds == y_a).item() +
                                                (1 - float(lam)) * torch.sum(preds == y_b).item())
                        else:
                            outputs = model(inputs)
                            _, preds = torch.max(outputs, 1)

                            loss = criterion(outputs, labels)
                            running_corrects += torch.sum(preds == labels).item()

                        if phase == "train":
                            loss.backward()
                            optimizer.step()
                        
                running_loss += loss.item() * inputs.size(0)      

            epoch_loss = running_loss / dataset_size[phase]
            epoch_acc = running_corrects / dataset_size[phase]

            print(f"{phase} Loss :{epoch_loss:.4f} Acc: {epoch_acc:.4f}")     

            if phase == "train":
                history["train_loss"].append(epoch_loss)    
                history["train_acc"].append(epoch_acc)   
            else:
                history["val_loss"].append(epoch_loss)  
                history["val_acc"].append(epoch_acc)  

            if phase == "val" and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()              

    time_elapsed = time.time() - since
    print(f"Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s")
    print(f"Best val Acc: {best_acc:.4f}")

    model.load_state_dict(best_model_wts)
    return model, history

In [46]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def evaluate_model_metrics(model, dataloader, device):
    model.eval()
    preds_list, labels_list = [], []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            preds_list.extend(preds.cpu().numpy())
            labels_list.extend(labels.cpu().numpy())

    acc = accuracy_score(labels_list, preds_list)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels_list, preds_list, average="macro", zero_division=0
    )

    return {
        "accuracy": acc,
        "macro_precision": precision,
        "macro_recall": recall,
        "macro_f1": f1,
        "n": len(labels_list),
    }

In [47]:
def build_subgroup_folder(
    csv_path,
    images_root,
    out_root,
    filter_col,
    filter_value,        
    class_col="class",
    id_col="id",
    overwrite=True,
    copy_files=True    
):
    csv_path = Path(csv_path)
    images_root = Path(images_root)
    out_root = Path(out_root)

    df = pd.read_csv(csv_path)

    sub = df[df[filter_col] == filter_value].copy()
    subgroup_name = f"{filter_col}_{filter_value}"
    dst_root = out_root / subgroup_name

    if overwrite and dst_root.exists():
        shutil.rmtree(dst_root)
    dst_root.mkdir(parents=True, exist_ok=True)

    missing = 0
    written = 0

    for _, row in sub.iterrows():
        cls = str(row[class_col]).strip()
        fname = str(row[id_col]).strip()

        src = images_root / cls / fname
        if not src.exists():
            missing += 1
            continue

        dst_class_dir = dst_root / cls
        dst_class_dir.mkdir(parents=True, exist_ok=True)
        dst = dst_class_dir / fname

        if copy_files:
            shutil.copy2(src, dst)
        else:
            if dst.exists():
                dst.unlink()
            dst.symlink_to(src.resolve())

        written += 1
        
    return dst_root


In [48]:
csv_path = "subgroups.csv"
images_root = input_path2
out_root = "subgroups"

low_light_root = build_subgroup_folder(csv_path, images_root, out_root,
                                       filter_col="lighting", filter_value=2,
                                       copy_files=True)

good_light_root = build_subgroup_folder(csv_path, images_root, out_root,
                                        filter_col="lighting", filter_value=1,
                                        copy_files=True)

background_tree_root = build_subgroup_folder(csv_path, images_root, out_root,
                                       filter_col="background", filter_value="tree",
                                       copy_files=True)

background_wall_root = build_subgroup_folder(csv_path, images_root, out_root,
                                       filter_col="background", filter_value="wall",
                                       copy_files=True)

background_white_root = build_subgroup_folder(csv_path, images_root, out_root,
                                       filter_col="background", filter_value="white",
                                       copy_files=True)

background_floor_root = build_subgroup_folder(csv_path, images_root, out_root,
                                       filter_col="background", filter_value="floor",
                                       copy_files=True)  

near_view_root = build_subgroup_folder(csv_path, images_root, out_root,
                                       filter_col="view", filter_value=1,
                                       copy_files=True)

far_view_root = build_subgroup_folder(csv_path, images_root, out_root,
                                        filter_col="view", filter_value=2,
                                        copy_files=True)                                                                                                                   

In [49]:
def evaluate_all_subgroups(
    model,
    subgroups_root,       
    transform,            
    device,
    batch_size=32,
    num_workers=4,
    print_results=True
):

    subgroups_root = Path(subgroups_root)
    assert subgroups_root.exists(), f"Subgroups root not found: {subgroups_root}"

    results = []
    subgroup_dirs = sorted([p for p in subgroups_root.iterdir() if p.is_dir()])

    for sg_dir in subgroup_dirs:
        has_class_dirs = any([p.is_dir() for p in sg_dir.iterdir()])
        if not has_class_dirs:
            results.append({
                "subgroup": sg_dir.name,
                "accuracy": np.nan,
                "macro_precision": np.nan,
                "macro_recall": np.nan,
                "macro_f1": np.nan,
                "n": 0,
                "note": "empty subgroup folder"
            })
            continue

        ds = datasets.ImageFolder(sg_dir, transform=transform)

        if len(ds) == 0:
            results.append({
                "subgroup": sg_dir.name,
                "accuracy": np.nan,
                "macro_precision": np.nan,
                "macro_recall": np.nan,
                "macro_f1": np.nan,
                "n": 0,
                "note": "no images"
            })
            continue

        loader = torch.utils.data.DataLoader(
            ds, batch_size=batch_size, shuffle=False, num_workers=num_workers
        )

        metrics = evaluate_model_metrics(model, loader, device)
        metrics["subgroup"] = sg_dir.name
        metrics["note"] = ""
        results.append(metrics)

        if print_results:
            print(f"{sg_dir.name:20s} | n={metrics['n']:4d} | "
                  f"acc={metrics['accuracy']:.4f} | macroF1={metrics['macro_f1']:.4f}")

    df = pd.DataFrame(results).sort_values(by="subgroup").reset_index(drop=True)
    return df

In [28]:
#RESNET-50

resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT) #Load pretrained model
resnet.fc = nn.Linear(resnet.fc.in_features, num_classes) #Replace last classifier layer
resnet = resnet.to(device)

optimizer = optim.SGD(resnet.parameters(), lr=0.001) #define optimizer

resnet, resnet_history = train_model(
    model=resnet,
    dataloaders=dataloaders,
    dataset_size=dataset_size,
    device=device,
    criterion=criterion_weighted,
    optimizer=optimizer,
    num_epochs=25,
    use_mixup=False,
    mixup_prob=0.5,
    mixup_alpha=0.4
)


Epoch 1/25
----------
train Loss :1.7872 Acc: 0.1887
val Loss :1.7691 Acc: 0.2032

Epoch 2/25
----------
train Loss :1.7384 Acc: 0.2947
val Loss :1.7137 Acc: 0.3745

Epoch 3/25
----------
train Loss :1.6961 Acc: 0.4121
val Loss :1.6596 Acc: 0.5060

Epoch 4/25
----------
train Loss :1.6556 Acc: 0.4953
val Loss :1.6109 Acc: 0.5976

Epoch 5/25
----------
train Loss :1.6140 Acc: 0.5527
val Loss :1.5797 Acc: 0.6653

Epoch 6/25
----------
train Loss :1.5681 Acc: 0.5973
val Loss :1.5045 Acc: 0.7211

Epoch 7/25
----------
train Loss :1.5154 Acc: 0.6038
val Loss :1.4555 Acc: 0.7450

Epoch 8/25
----------
train Loss :1.4664 Acc: 0.6236
val Loss :1.3833 Acc: 0.7610

Epoch 9/25
----------
train Loss :1.4233 Acc: 0.6444
val Loss :1.3649 Acc: 0.7291

Epoch 10/25
----------
train Loss :1.3714 Acc: 0.6568
val Loss :1.2840 Acc: 0.7450

Epoch 11/25
----------
train Loss :1.3146 Acc: 0.6642
val Loss :1.2224 Acc: 0.7649

Epoch 12/25
----------
train Loss :1.2737 Acc: 0.6657
val Loss :1.1630 Acc: 0.7410

E

In [57]:
print("Res-net ID metrics")
test_metrics = evaluate_model_metrics(resnet, dataloaders_photo["test"], device)
print(test_metrics)
print("Res-Net OOD metrics")
ood_metrics = evaluate_model_metrics(resnet, dataloaders_photo["test_01"], device)
print(ood_metrics)

Res-net ID metrics
{'accuracy': 0.8054474708171206, 'macro_precision': 0.7726845522898155, 'macro_recall': 0.7791504406640705, 'macro_f1': 0.7713021970082439, 'n': 257}
Res-Net OOD metrics
{'accuracy': 0.5627615062761506, 'macro_precision': 0.5843147165796183, 'macro_recall': 0.5625, 'macro_f1': 0.5630956405516339, 'n': 478}


In [51]:
transform_eval = data_transforms["val"]

df_subgroups = evaluate_all_subgroups(
    model=resnet,      
    subgroups_root="subgroups",
    transform=transform_eval,
    device=device,
    batch_size=32,
    num_workers=4,
    print_results=True
)

df_subgroups


background_floor     | n= 120 | acc=0.6083 | macroF1=0.5758
background_tree      | n= 120 | acc=0.4917 | macroF1=0.4808
background_wall      | n= 118 | acc=0.6356 | macroF1=0.6296
background_white     | n= 120 | acc=0.5167 | macroF1=0.5288
lighting_1           | n= 239 | acc=0.6067 | macroF1=0.6050
lighting_2           | n= 239 | acc=0.5188 | macroF1=0.5165
view_1               | n= 240 | acc=0.5625 | macroF1=0.5567
view_2               | n= 238 | acc=0.5630 | macroF1=0.5667


,accuracy,macro_precision,macro_recall,macro_f1,n,subgroup,note
0,0.608333,0.595732,0.608333,0.575806,120,background_floor,
1,0.491667,0.571825,0.491667,0.480798,120,background_tree,
2,0.635593,0.674481,0.636111,0.629641,118,background_wall,
3,0.516667,0.573967,0.516667,0.528848,120,background_white,
4,0.606695,0.616825,0.606303,0.605011,239,lighting_1,
5,0.518828,0.566485,0.518697,0.516467,239,lighting_2,
6,0.562500,0.584694,0.562500,0.556713,240,view_1,
7,0.563025,0.584650,0.562939,0.566749,238,view_2,


In [52]:
#RESNET-50 photo augmentations

resnet_aug_1 = models.resnet50(weights=models.ResNet50_Weights.DEFAULT) #Load pretrained model
resnet_aug_1.fc = nn.Linear(resnet_aug_1.fc.in_features, num_classes) #Replace last classifier layer
resnet_aug_1 = resnet_aug_1.to(device)

optimizer = optim.SGD(resnet_aug_1.parameters(), lr=0.001) #define optimizer

resnet_aug_1, resnet_history_aug_1 = train_model(
    model=resnet_aug_1,
    dataloaders=dataloaders_photo,
    dataset_size=dataset_size_photo,
    device=device,
    criterion=criterion_weighted,
    optimizer=optimizer,
    num_epochs=25,
    use_mixup=False,
    mixup_prob=0.5,
    mixup_alpha=0.4
)


Epoch 1/25
----------
train Loss :1.7935 Acc: 0.1872
val Loss :1.7676 Acc: 0.2151

Epoch 2/25
----------
train Loss :1.7519 Acc: 0.2754
val Loss :1.7240 Acc: 0.3227

Epoch 3/25
----------
train Loss :1.7170 Acc: 0.3596
val Loss :1.6705 Acc: 0.4462

Epoch 4/25
----------
train Loss :1.6781 Acc: 0.4235
val Loss :1.6297 Acc: 0.5498

Epoch 5/25
----------
train Loss :1.6384 Acc: 0.5007
val Loss :1.5802 Acc: 0.5737

Epoch 6/25
----------
train Loss :1.5920 Acc: 0.5617
val Loss :1.5305 Acc: 0.6175

Epoch 7/25
----------
train Loss :1.5565 Acc: 0.5914
val Loss :1.4906 Acc: 0.6414

Epoch 8/25
----------
train Loss :1.5089 Acc: 0.5835
val Loss :1.4255 Acc: 0.6454

Epoch 9/25
----------
train Loss :1.4618 Acc: 0.6107
val Loss :1.3714 Acc: 0.6813

Epoch 10/25
----------
train Loss :1.4204 Acc: 0.6424
val Loss :1.3602 Acc: 0.6853

Epoch 11/25
----------
train Loss :1.3934 Acc: 0.6191
val Loss :1.2643 Acc: 0.7012

Epoch 12/25
----------
train Loss :1.3401 Acc: 0.6365
val Loss :1.2403 Acc: 0.7131

E

In [56]:
print("Res-net ID metrics")
test_metrics = evaluate_model_metrics(resnet_aug_1, dataloaders_photo["test"], device)
print(test_metrics)
print("Res-Net OOD metrics")
ood_metrics = evaluate_model_metrics(resnet_aug_1, dataloaders_photo["test_01"], device)
print(ood_metrics)

Res-net ID metrics
{'accuracy': 0.7315175097276264, 'macro_precision': 0.69988556236434, 'macro_recall': 0.7136195941791351, 'macro_f1': 0.6997284882001963, 'n': 257}
Res-Net OOD metrics
{'accuracy': 0.5878661087866108, 'macro_precision': 0.6325503186124876, 'macro_recall': 0.5879807692307693, 'macro_f1': 0.5774128070370734, 'n': 478}


In [54]:
transform_eval = data_transforms["val"]

df_subgroups = evaluate_all_subgroups(
    model=resnet_aug_1,      
    subgroups_root="subgroups",
    transform=transform_eval,
    device=device,
    batch_size=32,
    num_workers=4,
    print_results=True
)

df_subgroups


background_floor     | n= 120 | acc=0.6417 | macroF1=0.6068
background_tree      | n= 120 | acc=0.5917 | macroF1=0.5807
background_wall      | n= 118 | acc=0.5763 | macroF1=0.5583
background_white     | n= 120 | acc=0.5417 | macroF1=0.5363
lighting_1           | n= 239 | acc=0.5941 | macroF1=0.5780
lighting_2           | n= 239 | acc=0.5816 | macroF1=0.5752
view_1               | n= 240 | acc=0.6208 | macroF1=0.6137
view_2               | n= 238 | acc=0.5546 | macroF1=0.5390


,accuracy,macro_precision,macro_recall,macro_f1,n,subgroup,note
0,0.641667,0.633554,0.641667,0.606805,120,background_floor,
1,0.591667,0.665936,0.591667,0.580720,120,background_tree,
2,0.576271,0.693355,0.577778,0.558293,118,background_wall,
3,0.541667,0.590424,0.541667,0.536331,120,background_white,
4,0.594142,0.646218,0.594124,0.578012,239,lighting_1,
5,0.581590,0.625870,0.581838,0.575192,239,lighting_2,
6,0.620833,0.668802,0.620833,0.613702,240,view_1,
7,0.554622,0.602522,0.554605,0.538991,238,view_2,


In [58]:
#RESNET-50 geo augmentations

resnet_aug_2 = models.resnet50(weights=models.ResNet50_Weights.DEFAULT) #Load pretrained model
resnet_aug_2.fc = nn.Linear(resnet_aug_2.fc.in_features, num_classes) #Replace last classifier layer
resnet_aug_2 = resnet_aug_2.to(device)

optimizer = optim.SGD(resnet_aug_2.parameters(), lr=0.001) #define optimizer

resnet_aug_2, resnet_history_aug_2 = train_model(
    model=resnet_aug_2,
    dataloaders=dataloaders_geo,
    dataset_size=dataset_size_geo,
    device=device,
    criterion=criterion_weighted,
    optimizer=optimizer,
    num_epochs=25,
    use_mixup=False,
    mixup_prob=0.5,
    mixup_alpha=0.4
)


Epoch 1/25
----------
train Loss :1.7795 Acc: 0.1823
val Loss :1.7565 Acc: 0.2629

Epoch 2/25
----------
train Loss :1.7456 Acc: 0.2858
val Loss :1.7143 Acc: 0.3625

Epoch 3/25
----------
train Loss :1.7095 Acc: 0.3947
val Loss :1.6791 Acc: 0.4661

Epoch 4/25
----------
train Loss :1.6837 Acc: 0.4443
val Loss :1.6317 Acc: 0.5418

Epoch 5/25
----------
train Loss :1.6377 Acc: 0.5102
val Loss :1.5825 Acc: 0.6056

Epoch 6/25
----------
train Loss :1.6064 Acc: 0.5433
val Loss :1.5409 Acc: 0.6494

Epoch 7/25
----------
train Loss :1.5676 Acc: 0.5765
val Loss :1.4944 Acc: 0.6653

Epoch 8/25
----------
train Loss :1.5235 Acc: 0.5810
val Loss :1.4791 Acc: 0.6693

Epoch 9/25
----------
train Loss :1.4764 Acc: 0.6033
val Loss :1.4226 Acc: 0.6972

Epoch 10/25
----------
train Loss :1.4316 Acc: 0.6221
val Loss :1.3447 Acc: 0.7012

Epoch 11/25
----------
train Loss :1.3850 Acc: 0.6275
val Loss :1.3038 Acc: 0.7092

Epoch 12/25
----------
train Loss :1.3361 Acc: 0.6563
val Loss :1.2686 Acc: 0.7410

E

In [62]:
print("Res-net Geo ID metrics")
test_metrics = evaluate_model_metrics(resnet_aug_2, dataloaders_geo["test"], device)
print(test_metrics)
print("Res-Net Geo OOD metrics")
ood_metrics = evaluate_model_metrics(resnet_aug_2, dataloaders_geo["test_01"], device)
print(ood_metrics)

Res-net Geo ID metrics
{'accuracy': 0.7587548638132295, 'macro_precision': 0.7434927800749688, 'macro_recall': 0.7253181760509082, 'macro_f1': 0.7297055692898495, 'n': 257}
Res-Net Geo OOD metrics
{'accuracy': 0.50418410041841, 'macro_precision': 0.5652927997355243, 'macro_recall': 0.5028311965811966, 'macro_f1': 0.4885215034156976, 'n': 478}


In [ ]:
transform_eval = data_transforms["val"]

df_subgroups = evaluate_all_subgroups(
    model=resnet_aug_2,      
    subgroups_root="subgroups",
    transform=transform_eval,
    device=device,
    batch_size=32,
    num_workers=4,
    print_results=True
)

df_subgroups


background_floor     | n= 120 | acc=0.6417 | macroF1=0.6068
background_tree      | n= 120 | acc=0.5917 | macroF1=0.5807
background_wall      | n= 118 | acc=0.5763 | macroF1=0.5583
background_white     | n= 120 | acc=0.5417 | macroF1=0.5363
lighting_1           | n= 239 | acc=0.5941 | macroF1=0.5780
lighting_2           | n= 239 | acc=0.5816 | macroF1=0.5752
view_1               | n= 240 | acc=0.6208 | macroF1=0.6137
view_2               | n= 238 | acc=0.5546 | macroF1=0.5390


,accuracy,macro_precision,macro_recall,macro_f1,n,subgroup,note
0,0.641667,0.633554,0.641667,0.606805,120,background_floor,
1,0.591667,0.665936,0.591667,0.580720,120,background_tree,
2,0.576271,0.693355,0.577778,0.558293,118,background_wall,
3,0.541667,0.590424,0.541667,0.536331,120,background_white,
4,0.594142,0.646218,0.594124,0.578012,239,lighting_1,
5,0.581590,0.625870,0.581838,0.575192,239,lighting_2,
6,0.620833,0.668802,0.620833,0.613702,240,view_1,
7,0.554622,0.602522,0.554605,0.538991,238,view_2,


In [68]:
#RESNET-50 mixed augmentations

resnet_mixed = models.resnet50(weights=models.ResNet50_Weights.DEFAULT) #Load pretrained model
resnet_mixed.fc = nn.Linear(resnet_mixed.fc.in_features, num_classes) #Replace last classifier layer
resnet_mixed = resnet_mixed.to(device)

optimizer = optim.SGD(resnet_mixed.parameters(), lr=0.001) #define optimizer

resnet_mixed, resnet_mixed_history = train_model(
    model=resnet_mixed,
    dataloaders=dataloaders_mix,
    dataset_size=dataset_size_mix,
    device=device,
    criterion=criterion_weighted,
    optimizer=optimizer,
    num_epochs=25,
    use_mixup=False,
    mixup_prob=0.5,
    mixup_alpha=0.4
)


Epoch 1/25
----------
train Loss :1.7819 Acc: 0.1833
val Loss :1.7714 Acc: 0.2231

Epoch 2/25
----------
train Loss :1.7628 Acc: 0.2472
val Loss :1.7466 Acc: 0.2948

Epoch 3/25
----------
train Loss :1.7417 Acc: 0.3244
val Loss :1.7177 Acc: 0.3705

Epoch 4/25
----------
train Loss :1.7198 Acc: 0.3853
val Loss :1.6838 Acc: 0.4661

Epoch 5/25
----------
train Loss :1.7001 Acc: 0.4156
val Loss :1.6591 Acc: 0.5100

Epoch 6/25
----------
train Loss :1.6747 Acc: 0.4720
val Loss :1.6481 Acc: 0.5538

Epoch 7/25
----------
train Loss :1.6547 Acc: 0.5042
val Loss :1.5962 Acc: 0.6494

Epoch 8/25
----------
train Loss :1.6329 Acc: 0.5310
val Loss :1.5720 Acc: 0.6853

Epoch 9/25
----------
train Loss :1.5996 Acc: 0.5468
val Loss :1.5349 Acc: 0.6853

Epoch 10/25
----------
train Loss :1.5713 Acc: 0.5661
val Loss :1.4900 Acc: 0.7131

Epoch 11/25
----------
train Loss :1.5485 Acc: 0.5646
val Loss :1.4880 Acc: 0.7012

Epoch 12/25
----------
train Loss :1.5181 Acc: 0.5780
val Loss :1.4363 Acc: 0.7092

E

In [69]:
print("Res-net Mix ID metrics")
test_metrics = evaluate_model_metrics(resnet_mixed, dataloaders_mix["test"], device)
print(test_metrics)
print("Res-Net Mix OOD metrics")
ood_metrics = evaluate_model_metrics(resnet_mixed, dataloaders_mix["test_01"], device)
print(ood_metrics)

Res-net Mix ID metrics
{'accuracy': 0.7003891050583657, 'macro_precision': 0.698989898989899, 'macro_recall': 0.6796833367493339, 'macro_f1': 0.6804657519945038, 'n': 257}
Res-Net Mix OOD metrics
{'accuracy': 0.5774058577405857, 'macro_precision': 0.5740276025870498, 'macro_recall': 0.5764957264957264, 'macro_f1': 0.5737660243233843, 'n': 478}


In [70]:
transform_eval = data_transforms["val"]

df_subgroups = evaluate_all_subgroups(
    model=resnet_mixed,      
    subgroups_root="subgroups",
    transform=transform_eval,
    device=device,
    batch_size=32,
    num_workers=4,
    print_results=True
)

df_subgroups


background_floor     | n= 120 | acc=0.5667 | macroF1=0.5258
background_tree      | n= 120 | acc=0.5333 | macroF1=0.5360
background_wall      | n= 118 | acc=0.5763 | macroF1=0.5779
background_white     | n= 120 | acc=0.6333 | macroF1=0.6346
lighting_1           | n= 239 | acc=0.5900 | macroF1=0.5821
lighting_2           | n= 239 | acc=0.5649 | macroF1=0.5665
view_1               | n= 240 | acc=0.5958 | macroF1=0.5912
view_2               | n= 238 | acc=0.5588 | macroF1=0.5544


,accuracy,macro_precision,macro_recall,macro_f1,n,subgroup,note
0,0.566667,0.554080,0.566667,0.525849,120,background_floor,
1,0.533333,0.585337,0.533333,0.535982,120,background_tree,
2,0.576271,0.615405,0.573148,0.577850,118,background_wall,
3,0.633333,0.663034,0.633333,0.634642,120,background_white,
4,0.589958,0.582479,0.588889,0.582054,239,lighting_1,
5,0.564854,0.571248,0.564103,0.566461,239,lighting_2,
6,0.595833,0.596017,0.595833,0.591234,240,view_1,
7,0.558824,0.557616,0.557675,0.554360,238,view_2,


In [18]:
del resnet
torch.cuda.empty_cache()

In [ ]:
#Efficientnet

efficient_net = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT) #Load pretrained model

in_features = efficient_net.classifier[1].in_features #Replace last classifier layer
efficient_net.classifier[1] = nn.Linear(in_features, num_classes)

efficient_net = efficient_net.to(device)

criterion = nn.CrossEntropyLoss() #define loss
optimizer = optim.SGD(efficient_net.parameters(), lr=0.001) #define optimizer

efficient_net, efficient_net_history = train_model(
    model=efficient_net,
    dataloaders=dataloaders,
    dataset_size=dataset_size,
    device=device,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=25
)

In [ ]:
print("Efficient_net test metrics")
evaluate_model(efficient_net, dataloaders["test"], device, image_datasets["train"].classes)
print("Efficient_net test_01 metrics")
evaluate_model(efficient_net, dataloaders["test_01"], device, image_datasets["train"].classes)


In [ ]:
del efficient_net
torch.cuda.empty_cache()

In [ ]:
#densenet

densenet = models.densenet121(weights=models.DenseNet121_Weights.DEFAULT) #Load pretrained model
densenet.classifier = nn.Linear(densenet.classifier.in_features, num_classes)
densenet = densenet.to(device)

criterion = nn.CrossEntropyLoss() #define loss
optimizer = optim.SGD(densenet.parameters(), lr=0.001) #define optimizer

densenet, densenet_history = train_model(
    model=densenet,
    dataloaders=dataloaders,
    dataset_size=dataset_size,
    device=device,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=25
)

In [12]:
print("densenet test metrics")
evaluate_model(densenet, dataloaders["test"], device, image_datasets["train"].classes)
print("densenet test_01 metrics")
evaluate_model(densenet, dataloaders["test_01"], device, image_datasets["train"].classes)

densenet test metrics
Accuracy:  0.9066
Precision: 0.9172
Recall:    0.8697
F1-score:  0.8849

Per-class breakdown:
              precision    recall  f1-score   support

   cardboard       0.95      0.98      0.96        41
       glass       0.85      0.86      0.85        51
       metal       0.84      0.88      0.86        41
       paper       0.97      0.98      0.98        60
     plastic       0.90      0.92      0.91        49
       trash       1.00      0.60      0.75        15

    accuracy                           0.91       257
   macro avg       0.92      0.87      0.88       257
weighted avg       0.91      0.91      0.90       257

densenet test_01 metrics
Accuracy:  0.4270
Precision: 0.8017
Recall:    0.4294
F1-score:  0.4185

Per-class breakdown:
              precision    recall  f1-score   support

   cardboard       0.24      1.00      0.39        15
       glass       0.82      0.64      0.72        14
       metal       1.00      0.27      0.42        15
     

In [ ]:
del densenet
torch.cuda.empty_cache()


In [15]:
# ConvNeXt-Base
convnext_base = models.convnext_tiny(weights=models.ConvNeXt_Tiny_Weights.IMAGENET1K_V1)
in_features = convnext_base.classifier[2].in_features
convnext_base.classifier[2] = nn.Linear(in_features, num_classes)
convnext_base = convnext_base.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(convnext_base.parameters(), lr=0.001)  

convnext_base, convnext_base_history = train_model(
    model=convnext_base,
    dataloaders=dataloaders,
    dataset_size=dataset_size,
    device=device,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=25
)

1.6%

Downloading: "https://download.pytorch.org/models/convnext_tiny-983f1562.pth" to /home/sera/.cache/torch/hub/checkpoints/convnext_tiny-983f1562.pth


100.0%


Epoch 1/25
----------
train Loss :1.5678 Acc: 0.4255
val Loss :1.3140 Acc: 0.5817

Epoch 2/25
----------
train Loss :1.1305 Acc: 0.7083
val Loss :0.9192 Acc: 0.7849

Epoch 3/25
----------
train Loss :0.8742 Acc: 0.7940
val Loss :0.7776 Acc: 0.8247

Epoch 4/25
----------
train Loss :0.7217 Acc: 0.8143
val Loss :0.6549 Acc: 0.8327

Epoch 5/25
----------
train Loss :0.6075 Acc: 0.8455
val Loss :0.5063 Acc: 0.8805

Epoch 6/25
----------
train Loss :0.5324 Acc: 0.8623
val Loss :1.1671 Acc: 0.4940

Epoch 7/25
----------
train Loss :0.5099 Acc: 0.8613
val Loss :0.4446 Acc: 0.8805

Epoch 8/25
----------
train Loss :0.4330 Acc: 0.8920
val Loss :0.4732 Acc: 0.8845

Epoch 9/25
----------
train Loss :0.3919 Acc: 0.9000
val Loss :0.3862 Acc: 0.8964

Epoch 10/25
----------
train Loss :0.3516 Acc: 0.9099
val Loss :0.5459 Acc: 0.8406

Epoch 11/25
----------
train Loss :0.3484 Acc: 0.9104
val Loss :0.2957 Acc: 0.9323

Epoch 12/25
----------
train Loss :0.2991 Acc: 0.9292
val Loss :0.5274 Acc: 0.8247

E

In [16]:
print("ConvNext test metrics")
evaluate_model(convnext_base, dataloaders["test"], device, image_datasets["train"].classes)
print("convnext_base test_01 metrics")
evaluate_model(convnext_base, dataloaders["test_01"], device, image_datasets["train"].classes)

ConvNext test metrics
Accuracy:  0.9183
Precision: 0.9298
Recall:    0.8988
F1-score:  0.9091

Per-class breakdown:
              precision    recall  f1-score   support

   cardboard       0.95      0.98      0.96        41
       glass       0.95      0.82      0.88        51
       metal       0.87      0.95      0.91        41
       paper       0.97      0.95      0.96        60
     plastic       0.84      0.96      0.90        49
       trash       1.00      0.73      0.85        15

    accuracy                           0.92       257
   macro avg       0.93      0.90      0.91       257
weighted avg       0.92      0.92      0.92       257

convnext_base test_01 metrics
Accuracy:  0.3708
Precision: 0.5576
Recall:    0.3698
F1-score:  0.3517

Per-class breakdown:
              precision    recall  f1-score   support

   cardboard       0.23      1.00      0.38        15
       glass       1.00      0.29      0.44        14
       metal       0.83      0.33      0.48        15


In [ ]:
del convnext_base
torch.cuda.empty_cache()

In [2]:
import pandas as pd

df = pd.read_csv("subgroups.csv")

for col in ["lighting", "background", "view"]:
    print(f"\n{col.upper()}")
    print(df[col].value_counts())



LIGHTING
lighting
1    239
2    239
Name: count, dtype: int64

BACKGROUND
background
tree     120
white    120
floor    120
wall     118
Name: count, dtype: int64

VIEW
view
1    240
2    238
Name: count, dtype: int64
